In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

/Users/dingzeyuan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/dingzeyuan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
#loading data
df = pd.read_csv('MBTI_500.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'MBTI_500.csv'

In [3]:
missing_value = df.isnull().sum()
missing_value

posts    0
type     0
dtype: int64

In [4]:
type_distribution = df['type'].value_counts()
type_distribution

type
INTP    24961
INTJ    22427
INFJ    14963
INFP    12134
ENTP    11725
ENFP     6167
ISTP     3424
ENTJ     2955
ESTP     1986
ENFJ     1534
ISTJ     1243
ISFP      875
ISFJ      650
ESTJ      482
ESFP      360
ESFJ      181
Name: count, dtype: int64

In [5]:
# Get the mini type number
min_count = df['type'].value_counts().min()

In [6]:
# choose the mini number for all MBTI
df_balanced = pd.concat([
    group.sample(n=min_count, random_state=40)
    for _, group in df.groupby('type')
]).reset_index(drop=True)

In [27]:
df_balanced.head()

,posts,type
0,look mine shame still people bid end yea hear ...,ENFJ
1,intimacy skyrocket problem internet character ...,ENFJ
2,good luck patient people issue provide nasty m...,ENFJ
3,others obnoxiously weak te want someone eat so...,ENFJ
4,switch see blooper end knock computer monitor ...,ENFJ


In [8]:
X_raw = df_balanced['posts']
y = df_balanced['type']

vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words='english', 
    max_features=1000,        
    ngram_range=(1, 2)  
)

X = vectorizer.fit_transform(X_raw)

print(f"Martix shape: {X.shape}")

Martix shape: (2896, 1000)


#### # 朴素贝叶斯部分

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [19]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

y_pred = nb_model.predict(X_test)

print("Accuracy")
print(classification_report(y_test, y_pred))

Accuracy
              precision    recall  f1-score   support

        ENFJ       0.70      0.43      0.53        37
        ENFP       0.50      0.62      0.55        34
        ENTJ       0.76      0.58      0.66        33
        ENTP       0.71      0.51      0.60        39
        ESFJ       0.68      0.69      0.68        36
        ESFP       0.62      0.82      0.70        39
        ESTJ       1.00      0.86      0.92        49
        ESTP       0.92      0.88      0.90        25
        INFJ       0.31      0.70      0.43        30
        INFP       0.56      0.61      0.58        38
        INTJ       0.58      0.43      0.49        35
        INTP       0.57      0.62      0.59        34
        ISFJ       0.73      0.64      0.68        42
        ISFP       0.62      0.58      0.60        36
        ISTJ       0.78      0.70      0.74        40
        ISTP       0.86      0.73      0.79        33

    accuracy                           0.65       580
   macro avg     

In [17]:
def predict_mbti(word_or_phrase):
    X_input = vectorizer.transform([word_or_phrase])
    prediction = nb_model.predict(X_input)[0]
    return prediction

print(predict_mbti("tell"))
print(predict_mbti("emotional"))
print(predict_mbti("introvert"))

ENFP
ENFJ
ISTJ


#### BERT

In [24]:
import torch
print(torch.__version__)  
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

1.13.1


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
module 'torch' has no attribute 'compiler'

In [7]:
#loading data
df = pd.read_csv('MBTI 500.csv')
df.head()

,posts,type
0,know intj tool use interaction people excuse a...,INTJ
1,rap music ehh opp yeah know valid well know fa...,INTJ
2,preferably p hd low except wew lad video p min...,INTJ
3,drink like wish could drink red wine give head...,INTJ
4,space program ah bad deal meing freelance max ...,INTJ


In [8]:
missing_value = df.isnull().sum()
missing_value

posts    0
type     0
dtype: int64

In [9]:
type_distribution = df['type'].value_counts()
type_distribution

type
INTP    24961
INTJ    22427
INFJ    14963
INFP    12134
ENTP    11725
ENFP     6167
ISTP     3424
ENTJ     2955
ESTP     1986
ENFJ     1534
ISTJ     1243
ISFP      875
ISFJ      650
ESTJ      482
ESFP      360
ESFJ      181
Name: count, dtype: int64

In [10]:
min_count = df['type'].value_counts().min()
df_balanced = pd.concat([
    group.sample(n=min_count, random_state=40)
    for _, group in df.groupby('type')
]).reset_index(drop=True)

In [11]:
X_bert = df_balanced['posts'].tolist()
y_bert = df_balanced['type'].tolist()

In [13]:
label_encoder = LabelEncoder()
y_bert_encoded = label_encoder.fit_transform(y_bert)
num_labels = len(label_encoder.classes_)

NameError: name 'LabelEncoder' is not defined

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    X_bert, y_bert_encoded, test_size=0.2, stratify=y_bert_encoded, random_state=42
)

In [11]:
train_encodings = tokenizer(X_train_texts, truncation=True, padding=True, max_length=64, return_tensors='pt')
test_encodings = tokenizer(X_test_texts, truncation=True, padding=True, max_length=64, return_tensors='pt')

In [12]:
train_dataset = TensorDataset(
    train_encodings['input_ids'],
    train_encodings['attention_mask'],
    torch.tensor(y_train)
)

test_dataset = TensorDataset(
    test_encodings['input_ids'],
    test_encodings['attention_mask'],
    torch.tensor(y_test)
)

In [13]:
print(train_dataset)
print(test_dataset)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:
print(device)

cpu


In [16]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [17]:
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(2):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

    for batch in progress_bar:
        b_input_ids, b_attn_mask, b_labels = [x.to(device) for x in batch]
        optimizer.zero_grad()
        outputs = model(
            input_ids=b_input_ids,
            attention_mask=b_attn_mask,
            labels=b_labels
        )
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix(loss=loss.item())

    print(f" Epoch {epoch+1} - Total Loss: {total_loss:.4f}")

C:\Users\Dell\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


✅ Epoch 1 完成 - Total Loss: 403.7739


✅ Epoch 2 完成 - Total Loss: 386.4773


In [18]:
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
    for batch in test_loader:
        b_input_ids, b_attn_mask, b_labels = [x.to(device) for x in batch]
        outputs = model(input_ids=b_input_ids, attention_mask=b_attn_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(b_labels.cpu().numpy())

In [19]:
macro_f1 = f1_score(y_true, y_pred, average='macro')
weighted_f1 = f1_score(y_true, y_pred, average='weighted')

print(f"\nMacro F1 Score:    {macro_f1:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")


Macro F1 Score:    0.1250
Weighted F1 Score: 0.1254
